In [1]:
import folium
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from folium.plugins import MarkerCluster

OVERPASS_URL = "http://overpass-api.de/api/interpreter"
DEFAULT_LOCATION = [-42.877250470859501, 147.29182659077199]
ZOOM_START = 15
RADIUS = 10000

def fetch_amenities_from_osm(latitude, longitude, amenity_type="restaurant", radius=RADIUS):
    query = f"""
    [out:json];
    (
        node["amenity"="{amenity_type}"](around:{radius},{latitude},{longitude});
        way["amenity"="{amenity_type}"](around:{radius},{latitude},{longitude});
        relation["amenity"="{amenity_type}"](around:{radius},{latitude},{longitude});
    );
    out center;
    """
    response = requests.get(OVERPASS_URL, params={'data': query})
    return response.json()['elements'] if response.status_code == 200 else []

def style_function(color):
    def function(feature):
        return {'fillColor': color, 'color': color}
    return function



def add_markers_to_cluster(cluster, amenities, color, icon, default_name):
    for amenity in amenities:
        if 'lat' in amenity and 'lon' in amenity:
            folium.Marker(
                location=[amenity['lat'], amenity['lon']],
                icon=folium.Icon(color=color, icon=icon, prefix='fa'),
                popup=amenity['tags'].get('name', default_name)
            ).add_to(cluster)



def add_tree_markers_to_cluster(cluster, geojson_data, default_name="Tree"):
    for feature in geojson_data['features']:
        # Check if the feature has a geometry and it's not None
        if 'geometry' in feature and feature['geometry'] is not None:
            coordinates = feature['geometry']['coordinates']
            folium.Marker(
                location=[coordinates[1], coordinates[0]],  # GeoJSON has lon,lat; Folium uses lat,lon
                popup=default_name,
                icon=folium.Icon(icon="tree", prefix='fa', color="green")  # Using FontAwesome tree icon
            ).add_to(cluster)




def load_geojson(file_path):
    with open(file_path) as f:
        return json.load(f)

# List of data files and their associated colors and names for layer control
data_files_with_details = [
    ('data/Building_Footprints.geojson', 'black', 'Building Footprints'),
    ('data/C12_0_Flood_Prone_Hazard_Areas_Code.geojson', 'blue', 'Flood Prone Areas'),
    ('data/Bushfire prone locations.geojson', 'orange', 'Bushfire Prone Areas'),
    ('data/Hobart_Interim_Planning_Scheme_2015_Overlays.geojson', 'orange', 'Historical buildings')
]


m = folium.Map(location=DEFAULT_LOCATION, zoom_start=ZOOM_START)

for data_file, color, name in data_files_with_details:
    folium.GeoJson(load_geojson(data_file), style_function=style_function(color), name=name).add_to(m)



amenities_data = [
    ("hospital", "Hospitals", "green", "plus", "Unknown Hospital"),
    ("school", "Schools", "blue", "graduation-cap", "Unknown School"),
    ("shop", "Post Office", "red", "shopping-cart", "Unknown Supermarket")
]

for amenity, cluster_name, color, icon, default_name in amenities_data:
    fetched_amenities = fetch_amenities_from_osm(*DEFAULT_LOCATION, amenity_type=amenity)
    cluster = MarkerCluster(name=cluster_name).add_to(m)
    add_markers_to_cluster(cluster, fetched_amenities, color, icon, default_name)


trees_geojson = load_geojson('data/TreesWithAssetID.geojson')
tree_cluster = MarkerCluster(name="Trees").add_to(m)
add_tree_markers_to_cluster(tree_cluster, trees_geojson)


# Add Layer Control to the map
folium.LayerControl().add_to(m)
m.save('map_with_overlay.html')
m
